In [13]:
import torch
import sys
sys.path.append("../src")
sys.path.append("../src/models")
from train import getModel, prepare_dataset
from experiments import experiments
import argparse
import tqdm
import numpy as np
from datasets.ConcatDataset import ConcatDataset
import pandas as pd
import geopandas as gpd
import sklearn.metrics
import os
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects

figdir="/home/marc/projects/ISPRS_CropTypeMapping/images/regions"

### Evaluate pretrained model with one dataloader

In [23]:
def get_run(seed,experiment, root):
    log = pd.read_csv(os.path.join(root,str(seed),experiment,"log.csv"))
    test = log.loc[log["mode"]=="test"]
    return test.sort_values(by="kappa", ascending=False).iloc[0]

def table_row(means, stds,sigmainterval=1):
    line = list()
    for mean, std in zip(means, stds):
        line.append(f"{mean:.2f}" + "$^{\pm" + f"{std*sigmainterval:.2f}" + "}$")
    return " & ".join(line) + " \\\ "

def get_acc_kappa(exp,root="/data/isprs/holl"):
    isprs_gaf_rnn_holl = [get_run(seed,exp[0],root=root) for seed in [0,1,2]]
    isprs_gaf_transformer_holl = [get_run(seed,exp[1],root=root) for seed in [0,1,2]]
    isprs_gaf_msresnet_holl = [get_run(seed,exp[2],root=root) for seed in [0,1,2]]
    isprs_gaf_tempcnn_holl = [get_run(seed,exp[3],root=root) for seed in [0,1,2]]

    isprs_gaf_rnn_holl_kappa = np.array([run.kappa for run in isprs_gaf_rnn_holl])
    isprs_gaf_rnn_holl_accuracy = np.array([run.accuracy for run in isprs_gaf_rnn_holl])
    isprs_gaf_transformer_holl_kappa = np.array([run.kappa for run in isprs_gaf_transformer_holl])
    isprs_gaf_transformer_holl_accuracy = np.array([run.accuracy for run in isprs_gaf_transformer_holl])
    isprs_gaf_msresnet_holl_kappa = np.array([run.kappa for run in isprs_gaf_msresnet_holl])
    isprs_gaf_msresnet_holl_accuracy = np.array([run.accuracy for run in isprs_gaf_msresnet_holl])
    isprs_gaf_tempcnn_holl_kappa = np.array([run.kappa for run in isprs_gaf_tempcnn_holl])
    isprs_gaf_tempcnn_holl_accuracy = np.array([run.accuracy for run in isprs_gaf_tempcnn_holl])

    hollgaf_kappa = [m.mean() for m in [isprs_gaf_rnn_holl_kappa,isprs_gaf_transformer_holl_kappa,isprs_gaf_msresnet_holl_kappa,isprs_gaf_tempcnn_holl_kappa]]
    hollgaf_kappa_std = [m.std() for m in [isprs_gaf_rnn_holl_kappa,isprs_gaf_transformer_holl_kappa,isprs_gaf_msresnet_holl_kappa,isprs_gaf_tempcnn_holl_kappa]]

    hollgaf_acc = [m.mean() for m in [isprs_gaf_rnn_holl_accuracy,isprs_gaf_transformer_holl_accuracy,isprs_gaf_msresnet_holl_accuracy,isprs_gaf_tempcnn_holl_accuracy]]
    hollgaf_acc_std = [m.std() for m in [isprs_gaf_rnn_holl_accuracy,isprs_gaf_transformer_holl_accuracy,isprs_gaf_msresnet_holl_accuracy,isprs_gaf_tempcnn_holl_accuracy]]
    return hollgaf_kappa, hollgaf_kappa_std, hollgaf_acc, hollgaf_acc_std

def fill_table(rows,desc="ov.acc"):
    return r"""\begin{tabular}{lllcccc}
\toprule
"""+desc+r""" & trained on & tested on & \rnn & \transformer & \msresnet & \tempcnn \\
\cmidrule(lr){4-4}\cmidrule(lr){5-5}\cmidrule(lr){6-6}\cmidrule(lr){7-7}
    """+rows+r"""
\bottomrule
\end{tabular}"""

def make_table(exp1,exp1_std,exp2,exp2_std,exp3,exp3_std,desc, train_on, test_on):

    line1 = f"\tprocessed & {train_on[0]} & {test_on[0]} & "+table_row(exp1,exp1_std,sigmainterval=1)
    line2 = f"\traw       & {train_on[1]} & {test_on[1]} & "+table_row(exp2,exp2_std,sigmainterval=1)
    line3 = f"\traw       & {train_on[2]} & {test_on[2]} & "+table_row(exp3,exp3_std,sigmainterval=1)
    rows = "\n".join([line1,line2,line3])
    return fill_table(rows,desc)

## Hollfeld experiment

Preprocessed model trained on HOLL
raw model either trained on HOLL or KRUM+NOWA+HOLL

## 23 classes

In [31]:
get_run(2,"isprs_tum_rnn_holl", root)

Unnamed: 0                1
epoch                     1
iteration               NaN
mode                   test
accuracy           0.470997
loss                2.08724
mean_accuracy     0.0434783
mean_recall       0.0434783
mean_precision    0.0204781
mean_f1           0.0278425
kappa                     0
Name: 1, dtype: object

In [33]:
root = "/data/isprs/holl"
tableroot = "/home/marc/projects/ISPRS_CropTypeMapping/tables/holl"

exp=["isprs_gaf_rnn_holl", "isprs_gaf_transformer_holl", "isprs_gaf_msresnet_holl", "isprs_gaf_tempcnn_holl"]
hollgaf_kappa, hollgaf_kappa_std, hollgaf_acc, hollgaf_acc_std = get_acc_kappa(exp,root=root)
exp=["isprs_tum_rnn_all","isprs_tum_transformer_all","isprs_tum_msresnet_all", "isprs_tum_tempcnn_all"]
alltum_kappa, alltum_kappa_std, alltum_acc, alltum_acc_std = get_acc_kappa(exp,root=root)
exp = ["isprs_tum_rnn_holl", "isprs_tum_transformer_holl", "isprs_tum_msresnet_holl", "isprs_tum_tempcnn_holl"]
holltum_kappa, holltum_kappa_std, holltum_acc, holltum_acc_std = get_acc_kappa(exp,root=root)

train_on = ["Hollfeld","Hollfeld+Krumbach+Bav.Forest","Hollfeld"]
test_on = ["Hollfeld","Hollfeld","Hollfeld"]

print("kappa")
print()
table=make_table(hollgaf_kappa,hollgaf_kappa_std,alltum_kappa,alltum_kappa_std, holltum_kappa,holltum_kappa_std,
                 "ov.acc", train_on, test_on)
print(table)
print()
fname = os.path.join(tableroot,"kappa.tex")
print(f"writing to {fname}")
print(table,file=open(fname,'w'))
print()
print("kappa")
print()
table = make_table(hollgaf_acc,hollgaf_acc_std,alltum_acc,alltum_acc_std, holltum_acc,holltum_acc_std,
                   "kappa", train_on, test_on)
print(table)
print()
fname = os.path.join(tableroot,"accuracy.tex")
print(f"writing to {fname}")
print(table,
      file=open(fname,'w'))
print()

kappa

\begin{tabular}{lllcccc}
\toprule
ov.acc & trained on & tested on & \rnn & \transformer & \msresnet & \tempcnn \\
\cmidrule(lr){4-4}\cmidrule(lr){5-5}\cmidrule(lr){6-6}\cmidrule(lr){7-7}
    	processed & Hollfeld & Hollfeld & 0.74$^{\pm0.00}$ & 0.76$^{\pm0.01}$ & 0.78$^{\pm0.01}$ & 0.78$^{\pm0.00}$ \\ 
	raw       & Hollfeld+Krumbach+Bav.Forest & Hollfeld & 0.63$^{\pm0.06}$ & 0.70$^{\pm0.02}$ & 0.66$^{\pm0.00}$ & 0.62$^{\pm0.02}$ \\ 
	raw       & Hollfeld & Hollfeld & 0.22$^{\pm0.31}$ & 0.64$^{\pm0.06}$ & 0.66$^{\pm0.00}$ & 0.62$^{\pm0.01}$ \\ 
\bottomrule
\end{tabular}

writing to /home/marc/projects/ISPRS_CropTypeMapping/tables/holl/kappa.tex

kappa

\begin{tabular}{lllcccc}
\toprule
kappa & trained on & tested on & \rnn & \transformer & \msresnet & \tempcnn \\
\cmidrule(lr){4-4}\cmidrule(lr){5-5}\cmidrule(lr){6-6}\cmidrule(lr){7-7}
    	processed & Hollfeld & Hollfeld & 0.80$^{\pm0.00}$ & 0.81$^{\pm0.00}$ & 0.83$^{\pm0.01}$ & 0.83$^{\pm0.00}$ \\ 
	raw       & Hollfeld+Krumbach

## 12 classes

In [25]:
if True:
    root = "/data/isprs/holl2"
    tableroot = "/home/marc/projects/ISPRS_CropTypeMapping/tables/holl2"

    exp=["isprs_gaf_rnn_holl", "isprs_gaf_transformer_holl", "isprs_gaf_msresnet_holl", "isprs_gaf_tempcnn_holl"]
    hollgaf_kappa, hollgaf_kappa_std, hollgaf_acc, hollgaf_acc_std = get_acc_kappa(exp,root=root)
    exp=["isprs_tum_rnn_all","isprs_tum_transformer_all","isprs_tum_msresnet_all", "isprs_tum_tempcnn_all"]
    alltum_kappa, alltum_kappa_std, alltum_acc, alltum_acc_std = get_acc_kappa(exp,root=root)
    exp = ["isprs_tum_rnn_holl", "isprs_tum_transformer_holl", "isprs_tum_msresnet_holl", "isprs_tum_tempcnn_holl"]
    holltum_kappa, holltum_kappa_std, holltum_acc, holltum_acc_std = get_acc_kappa(exp,root=root)

    print("kappa")
    print()
    table=make_table(hollgaf_kappa,hollgaf_kappa_std,alltum_kappa,alltum_kappa_std, holltum_kappa,holltum_kappa_std,desc="oc.acc")
    print(table)
    print()
    fname = os.path.join(tableroot,"kappa.tex")
    print(f"writing to {fname}")
    print(table,file=open(fname,'w'))
    print()
    print("kappa")
    print()
    table = make_table(hollgaf_acc,hollgaf_acc_std,alltum_acc,alltum_acc_std, holltum_acc,holltum_acc_std,desc="kappa")
    print(table)
    print()
    fname = os.path.join(tableroot,"accuracy.tex")
    print(f"writing to {fname}")
    print(table,
          file=open(fname,'w'))
    print()

FileNotFoundError: [Errno 2] File b'/data/isprs/holl2/1/isprs_tum_rnn_all/log.csv' does not exist: b'/data/isprs/holl2/1/isprs_tum_rnn_all/log.csv'

In [11]:
print("kappa")
print()
table=make_table(hollgaf_kappa,hollgaf_kappa_std,alltum_kappa,alltum_kappa_std, holltum_kappa,holltum_kappa_std,desc="oc.acc")
print(table)
print()
fname = os.path.join(tableroot,"kappa.tex")
print(f"writing to {fname}")
print(table,file=open(fname,'w'))
print()
print("kappa")
print()
table = make_table(hollgaf_acc,hollgaf_acc_std,alltum_acc,alltum_acc_std, holltum_acc,holltum_acc_std,desc="kappa")
print(table)
print()
fname = os.path.join(tableroot,"accuracy.tex")
print(f"writing to {fname}")
print(table,
      file=open(fname,'w'))
print()

kappa



TypeError: make_table() missing 2 required positional arguments: 'train_on' and 'test_on'

## Krumbach experiment

Preprocessed model trained on KRUM
raw model either trained on KRUM or KRUM+NOWA+HOLL

## 23 classes

In [22]:
root = "/data/isprs/krum"
tableroot = "/home/marc/projects/ISPRS_CropTypeMapping/tables/krum"

exp=["isprs_gaf_rnn_krum", "isprs_gaf_transformer_krum", "isprs_gaf_msresnet_krum", "isprs_gaf_tempcnn_krum"]
krumgaf_kappa, krumgaf_kappa_std, krumgaf_acc, krumgaf_acc_std = get_acc_kappa(exp,root=root)
exp=["isprs_tum_rnn_allkrum","isprs_tum_transformer_allkrum","isprs_tum_msresnet_allkrum", "isprs_tum_tempcnn_allkrum"]
alltum_kappa, alltum_kappa_std, alltum_acc, alltum_acc_std = get_acc_kappa(exp,root=root)
exp = ["isprs_tum_rnn_krum", "isprs_tum_transformer_krum", "isprs_tum_msresnet_krum", "isprs_tum_tempcnn_krum"]
krumtum_kappa, krumtum_kappa_std, krumtum_acc, krumtum_acc_std = get_acc_kappa(exp,root=root)

train_on = ["Krumbach","Krumbach+Hollfeld+Bav.Forest","Krumbach"]
test_on = ["Krumbach","Krumbach","Krumbach"]

print("kappa")
print()
table = make_table(krumgaf_kappa,krumgaf_kappa_std,alltum_kappa,alltum_kappa_std, krumtum_kappa,krumtum_kappa_std,
                   "kappa",train_on,test_on)
print(table)
print()
fname = os.path.join(tableroot,"kappa.tex")
print(f"writing to {fname}")
print(table,
      file=open(fname,'w'))
print()
print("accuracy")
print()
table=make_table(krumgaf_acc,krumgaf_acc_std,alltum_acc,alltum_acc_std, krumtum_acc,krumtum_acc_std,
                 "ov.acc",train_on,test_on)
print(table)
print()
fname = os.path.join(tableroot,"accuracy.tex")
print(f"writing to {fname}")
print(table,
      file=open(fname,'w'))
print()

kappa

\begin{tabular}{lllcccc}
\toprule
kappa & trained on & tested on & \rnn & \transformer & \msresnet & \tempcnn \\
\cmidrule(lr){4-4}\cmidrule(lr){5-5}\cmidrule(lr){6-6}\cmidrule(lr){7-7}
    	processed & Krumbach & Krumbach & 0.86$^{\pm0.01}$ & 0.87$^{\pm0.00}$ & 0.73$^{\pm0.13}$ & 0.87$^{\pm0.02}$ \\ 
	raw       & Krumbach+Hollfeld+Bav.Forest & Krumbach & 0.80$^{\pm0.01}$ & 0.77$^{\pm0.04}$ & 0.78$^{\pm0.02}$ & 0.74$^{\pm0.03}$ \\ 
	raw       & Krumbach & Krumbach & 0.78$^{\pm0.03}$ & 0.78$^{\pm0.04}$ & 0.69$^{\pm0.10}$ & 0.78$^{\pm0.01}$ \\ 
\bottomrule
\end{tabular}

writing to /home/marc/projects/ISPRS_CropTypeMapping/tables/krum/kappa.tex

accuracy

\begin{tabular}{lllcccc}
\toprule
ov.acc & trained on & tested on & \rnn & \transformer & \msresnet & \tempcnn \\
\cmidrule(lr){4-4}\cmidrule(lr){5-5}\cmidrule(lr){6-6}\cmidrule(lr){7-7}
    	processed & Krumbach & Krumbach & 0.91$^{\pm0.00}$ & 0.91$^{\pm0.00}$ & 0.80$^{\pm0.11}$ & 0.91$^{\pm0.02}$ \\ 
	raw       & Krumbach+Hollf

## 12 classes

In [233]:
if False:
    root = "/data/isprs/krum2"
    tableroot = "/home/marc/projects/ISPRS_CropTypeMapping/tables/krum2"

    exp=["isprs_gaf_rnn_krum", "isprs_gaf_transformer_krum", "isprs_gaf_msresnet_krum", "isprs_gaf_tempcnn_krum"]
    krumgaf_kappa, krumgaf_kappa_std, krumgaf_acc, krumgaf_acc_std = get_acc_kappa(exp,root=root)
    exp=["isprs_tum_rnn_allkrum","isprs_tum_transformer_allkrum","isprs_tum_msresnet_allkrum", "isprs_tum_tempcnn_allkrum"]
    alltum_kappa, alltum_kappa_std, alltum_acc, alltum_acc_std = get_acc_kappa(exp,root=root)
    exp = ["isprs_tum_rnn_krum", "isprs_tum_transformer_krum", "isprs_tum_msresnet_krum", "isprs_tum_tempcnn_krum"]
    krumtum_kappa, krumtum_kappa_std, krumtum_acc, krumtum_acc_std = get_acc_kappa(exp,root=root)

    print("kappa")
    print()
    table = make_table(krumgaf_kappa,krumgaf_kappa_std,alltum_kappa,alltum_kappa_std, krumtum_kappa,krumtum_kappa_std,desc="kappa")
    print(table)
    print()
    fname = os.path.join(tableroot,"kappa.tex")
    print(f"writing to {fname}")
    print(table,
          file=open(fname,'w'))
    print()
    print("accuracy")
    print()
    table=make_table(krumgaf_acc,krumgaf_acc_std,alltum_acc,alltum_acc_std, krumtum_acc,krumtum_acc_std,"ov.acc")
    print(table)
    print()
    fname = os.path.join(tableroot,"accuracy.tex")
    print(f"writing to {fname}")
    print(table,
          file=open(fname,'w'))
    print()